<img src="../assets/header_notebook.png" />
<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>ESA - Black Sea Deoxygenation Emulator</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# ----------
# Librairies
# ----------
import os
import sys
import cv2
import dawgz
import wandb
import xarray
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim

# Dawgz (jobs //)
from dawgz import job, schedule

# -------------------
# Librairies (Custom)
# -------------------
# Adding path to source folder to load custom modules
sys.path.append('/src')
sys.path.append('/src/debs/')
sys.path.insert(1, '/src/debs/')
sys.path.insert(1, '/scripts/')

# Moving to the .py directory
%cd src/debs/

## Loading libraries
from dataloader  import *
from dataset     import *
from losses      import *
from metrics     import *
from tools       import *
from training    import *

# -------
# Jupyter
# -------
%matplotlib inline
plt.rcParams.update({'font.size': 13})

# Making sure modules are reloaded when modified
%reload_ext autoreload
%autoreload 2

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Scripts</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# Training a neural network (using a given configuration)
%run __training.py --config local

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Dataloader</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# Libraries
import time
import wandb
import xarray
import calendar
import numpy as np

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LinearLR

# Custom libraries
from metrics                import *
from tools                  import *
from losses                 import *
from dataset                import BlackSea_Dataset
from dataloader             import BlackSea_Dataloader
from neural_networks.loader import load_neural_network

# -------------—---------
#     Initialization
# -------------—---------
#
# Information over terminal (1)
project_title(kwargs)

# Checking if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fixing random seed for reproducibility
np.random.seed(2701)
torch.manual_seed(2701)

# Loading configuration
project        = kwargs['Project']
mode           = kwargs['Mode']
window_input   = kwargs['Window (Inputs)']
window_output  = kwargs['Window (Outputs)']
frequencies    = kwargs['Frequencies']
architecture   = kwargs['Architecture']
learning_rate  = kwargs['Learning Rate']
batch_size     = kwargs['Batch Size']
nb_epochs      = kwargs['Epochs']
num_workers    = kwargs['Number of Workers']

# -------------—---------
#     Loading the data
# -----------------------
validation = BlackSea_Dataset("Validation")
test       = BlackSea_Dataset("Test")

# Extracting the output (used by AverageNET)
data_oxygen = validation.get_data(variable = "oxygen")

# Loading the mesh, masks and bathymetry
mesh                = test.get_mesh()
bs_mask             = test.get_mask(continental_shelf = False)
bs_mask_with_depth  = test.get_mask(continental_shelf = True)
bathymetry          = test.get_depth(unit = "meter")

# Hypoxia treshold
hypox_tresh = test.get_treshold(standardized = True)

In [ ]:
# Data loaders
BS_loader_validation = BlackSea_Dataloader(validation,
                                           window_input,
                                           window_output,
                                           frequencies,
                                           batch_size,
                                           num_workers,
                                           mesh,
                                           bs_mask,
                                           bs_mask_with_depth,
                                           bathymetry,
                                           random = True)

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# Parameters definition
kwargs = {
    "Project"           : "Test",
    "Mode"              : "disabled",
    "Window (Inputs)"   : 1,
    "Window (Outputs)"  : 10,
    "Frequencies"       : 16,
    "Architecture"      : "UNET",
    "Scaling"           : 1,
    "Learning Rate"     : 0.0001,
    "Batch Size"        : 32,
    "Epochs"            : 1,
    'Number of Workers' : 1,
}

In [ ]:
# Libraries
import time
import wandb
import xarray
import calendar
import numpy as np

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LinearLR

# Custom libraries
from metrics                import *
from tools                  import *
from losses                 import *
from dataset                import BlackSea_Dataset
from dataloader             import BlackSea_Dataloader
from neural_networks.loader import load_neural_network

# -------------—---------
#     Initialization
# -------------—---------
#
# Information over terminal (1)
project_title(kwargs)

# Checking if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fixing random seed for reproducibility
np.random.seed(2701)
torch.manual_seed(2701)

# Loading configuration
project        = kwargs['Project']
mode           = kwargs['Mode']
window_input   = kwargs['Window (Inputs)']
window_output  = kwargs['Window (Outputs)']
frequencies    = kwargs['Frequencies']
architecture   = kwargs['Architecture']
learning_rate  = kwargs['Learning Rate']
batch_size     = kwargs['Batch Size']
nb_epochs      = kwargs['Epochs']
num_workers    = kwargs['Number of Workers']

# -------------—---------
#     Loading the data
# -----------------------
validation = BlackSea_Dataset("Validation")
test       = BlackSea_Dataset("Test")

# Extracting the output (used by AverageNET)
data_oxygen = validation.get_data(variable = "oxygen")

# Loading the mesh, masks and bathymetry
mesh                = test.get_mesh()
bs_mask             = test.get_mask(continental_shelf = False)
bs_mask_with_depth  = test.get_mask(continental_shelf = True)
bathymetry          = test.get_depth(unit = "meter")

# Hypoxia treshold
hypox_tresh = test.get_treshold(standardized = True)

# Data loaders
BS_loader_validation = BlackSea_Dataloader(validation,
                                           window_input,
                                           window_output,
                                           frequencies,
                                           batch_size,
                                           num_workers,
                                           mesh,
                                           bs_mask,
                                           bs_mask_with_depth,
                                           bathymetry,
                                           random = True)

BS_loader_test = BlackSea_Dataloader(test,
                                     window_input,
                                     window_output,
                                     frequencies,
                                     batch_size,
                                     num_workers,
                                     mesh,
                                     bs_mask,
                                     bs_mask_with_depth,
                                     bathymetry,
                                     random = True)

# Creating the dataloaders
dataset_validation   = BS_loader_validation.get_dataloader()
dataset_test         = BS_loader_test.get_dataloader()

# -------------—--------------------
#     Neural Network & Training
# -------------—--------------------
# Initialization of the neural network
neural_net = load_neural_network(architecture = architecture, data_output = data_oxygen, device = device, kwargs = kwargs)
neural_net.to(device)

# Total number of available GPUs
num_gpus = torch.cuda.device_count()
print("Available GPUs: ", num_gpus)

# Total number of parameters
nn_params = neural_net.count_parameters()
print("Total number of parameters: ", nn_params/1e6, "M")

# Using multiple GPUS
neural_net = torch.nn.parallel.DataParallel(neural_net, device_ids=list(range(num_gpus)), dim=0)

# Loading the optimizer
optimizer  = optim.Adam(neural_net.parameters(), lr = learning_rate)

# Loading the scheduler
scheduler = LinearLR(optimizer, start_factor = 0.95, total_iters = nb_epochs)

In [ ]:
# WandB (1) - Initialization of the run
wandb.init(project = project, mode = mode, config = kwargs)
wandb.log({f"Total number of parameters": nn_params})

# Used to compute the total time left,
epoch_time = 0.0

# ------- Training Loop -------
for epoch in range(nb_epochs):

  # Timing the epoch (1)
  start = time.time()

  # Used to store and compute instantaneous training loss
  loss_training_total, loss_training_per_day, loss_training_index = 0.0, list(), 0

  # Training the neural network
  for x, t, y in dataset_test:

    # Pushing the data to the correct device
    x, y, t = x.to(device), y.to(device), t.to(device)

    # Forward pass
    pred = neural_net.forward(x, t)

    # Computing the training loss
    loss_training_batch_total, loss_training_batch_per_day = forecasting_loss(y_true = y,
                                                                              y_pred = pred,
                                                                                mask = bs_mask_with_depth)

    # Accumulating the total loss, storing losses per day and updating the number of training steps
    loss_training_total += loss_training_batch_total.item()
    loss_training_index += 1
    loss_training_per_day.append([l.item() for l in loss_training_batch_per_day])

    print("Training Loss:", loss_training_batch_total.item())

    # AverageNet : No optimization needed !
    if architecture == "AVERAGE":
        continue

    # Reseting the gradients
    optimizer.zero_grad()

    # Backward pass
    loss_training_batch_total.backward()

    # Optimizing the parameters
    optimizer.step()

    # Freeing the GPU
    x, y, t, pred = x.to("cpu"), y.to("cpu"), t.to("cpu"), pred.to("cpu")

    # WandB (2.1) - Sending information about the training results
    wandb.log({f"Training/Loss (T)": loss_training_batch_total.item()})
    wandb.log({f"Training/Loss (T, {i})": loss.item() for i, loss in enumerate(loss_training_batch_per_day)})

    # Freeing memory
    del x, y, t, pred, loss_training_batch_total, loss_training_batch_per_day
    break

  # WandB (2.2) - Sending information about the training results
  wandb.log({f"Training/Loss (Training): ": loss_training_total / loss_training_index})

  with torch.no_grad():

    # Used to store and compute instantaneous training loss
    loss_validation_total, loss_validation_per_day, loss_validation_index = 0.0, list(), 0

    # Used to store all predictions and ground truth
    validation_predictions, validation_ground_truth = None, None

    # Used to store temporal information
    time_days, time_months, time_years = list(), list(), list()

    # Validating the neural network
    for x, t, y in dataset_validation:

      # Pushing the data to the correct device
      x, y, t = x.to(device), y.to(device), t.to(device)

      # Forward pass
      pred = neural_net.forward(x, t)

      # Computing the validation loss
      loss_validation_batch_total, loss_validation_batch_per_day = forecasting_loss(y_true = y,
                                                                                    y_pred = pred,
                                                                                      mask = bs_mask_with_depth)

      # Accumulating the total loss, storing losses per day and updating the number of training steps
      loss_validation_total += loss_validation_batch_total.item()
      loss_validation_index += 1
      loss_validation_per_day.append([l.item() for l in loss_validation_batch_per_day])

      # Pushing everything back to the CPU
      x, y, t, pred = x.to("cpu"), y.to("cpu"), t.to("cpu"), pred.to("cpu")

      # WandB (3.1) - Sending information about the validation results
      wandb.log({f"Training/Loss (V)": loss_validation_batch_total.item()})
      wandb.log({f"Training/Loss (V, {i})": loss.item() for i, loss in enumerate(loss_validation_batch_per_day)})

      print("Validation Loss:", loss_validation_batch_total.item())

      # Storing results
      validation_predictions  = torch.cat((validation_predictions,  pred), dim = 0) if validation_predictions  is not None else pred
      validation_ground_truth = torch.cat((validation_ground_truth,    y), dim = 0) if validation_ground_truth is not None else y

      # Cleaning
      del x, y, t, pred, loss_validation_batch_total, loss_validation_batch_per_day
      torch.cuda.empty_cache()
      break

  # Updating the scheduler
  scheduler.step()

  # Timing the epoch (2)
  epoch_time = (time.time() - start)/3600

  # WandB (2.2) - Sending information about the training results
  wandb.log({f"Training/Loss (Validation): ": loss_validation_total / loss_validation_index,
             f"Training/Time Left (H)": epoch_time})

  # Computing the results
  #analyze(validation_ground_truth, validation_predictions, bs_mask_with_depth, validation, BS_loader_validation)

# Extracting the Neural Network back to CPU
neural_net.to("cpu")

# Finishing the Weight and Biases run
wandb.finish()